<a href="https://colab.research.google.com/github/giseletoledo/geradorvideocomIA/blob/main/criadorvideoAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install imagemagick

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts imagemagick-6-common
  imagemagick-6.q16 libdjvulibre-text libdjvulibre21 libfftw3-double3 libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10 libwmflite-0.2-7 netpbm
  poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x imagemagick-doc
  autotrace cups-bsd | lpr | lprng enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer
  povray radiance sane-utils texlive-base-bin transfig ufraw-batch libfftw3-bin libfftw3-dev
  inkscape poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic
  | fonts-ipafont-gothic fonts-arphic-ukai fonts-arphic-uming fonts-nan

In [ ]:
!pip install pillow moviepy gtts

In [ ]:
import io
import os
from PIL import Image
from moviepy.editor import ImageClip, TextClip, CompositeVideoClip, AudioFileClip
from gtts import gTTS
from IPython.display import Video, display

# Função para carregar uma imagem
def upload_image():
    from google.colab import files  # Se você estiver usando Colab
    uploaded = files.upload()
    image_path = next(iter(uploaded))  # Pega o primeiro arquivo
    return image_path

# Função para gerar áudio a partir do texto
def generate_audio_from_text(text, lang='pt-br'):  # Definido para português brasileiro
    tts = gTTS(text=text, lang=lang)
    audio_path = "audio.mp3"
    tts.save(audio_path)
    return audio_path

# Função para criar um vídeo com imagem, áudio e legenda
def create_video_with_caption_and_audio(image_path, text, audio_path, duration=5):
    # Criar um clipe de imagem
    image_clip = ImageClip(image_path).set_duration(duration)

    # Criar o clipe de texto (legenda)
    text_clip = TextClip(text, fontsize=40, color='white', bg_color='black', method='caption')
    text_clip = text_clip.set_position(('center', 'bottom')).set_duration(duration)

    # Adicionar o áudio ao clipe da imagem
    audio_clip = AudioFileClip(audio_path)
    image_clip = image_clip.set_audio(audio_clip)

    # Combinar a imagem e o texto (legenda) em um único clipe
    video = CompositeVideoClip([image_clip, text_clip])

    # Criar um buffer de bytes temporário
    buffer = io.BytesIO()

    # Escrever o vídeo no buffer
    video.write_videofile(buffer, codec='libx264', fps=24, temp_audiofile="temp-audio.m4a", remove_temp=True)

    buffer.seek(0)
    return buffer

# Passo 1: Upload da imagem
image_path = upload_image()

# Passo 2: Capturar a legenda
text_caption = input("Digite o texto da legenda: ")

# Passo 3: Gerar o áudio da legenda em português brasileiro
audio_path = generate_audio_from_text(text_caption)

# Passo 4: Criar o vídeo com imagem, áudio e legenda
video_buffer = create_video_with_caption_and_audio(image_path, text_caption, audio_path)

# Passo 5: Exibir o vídeo no Jupyter Notebook
display(Video(video_buffer.read(), embed=True, format='mp4'))

Saving WhatsApp Image 2024-09-28 at 23.21.11.jpeg to WhatsApp Image 2024-09-28 at 23.21.11 (2).jpeg
Digite o texto da legenda: Mulher empreendedora


OSError: MoviePy Error: creation of None failed because of the following error:

[Errno 2] No such file or directory: 'unset'.

.This error can be due to the fact that ImageMagick is not installed on your computer, or (for Windows users) that you didn't specify the path to the ImageMagick binary in file conf.py, or that the path you specified is incorrect

**Gerando video com áudio**

In [6]:
import io
from moviepy.editor import ImageClip, AudioFileClip, CompositeVideoClip
from gtts import gTTS
from PIL import Image, ImageDraw, ImageFont
from google.colab import files  # Se você estiver usando Colab
from IPython.display import Video, display

# Função para carregar uma imagem
def upload_image():
    uploaded = files.upload()  # Abre a caixa de upload
    image_path = next(iter(uploaded))  # Obtém o caminho do arquivo carregado
    return image_path

# Função para gerar áudio a partir do texto
def generate_audio_from_text(text, lang='pt'):  # Ajuste do idioma para 'pt'
    tts = gTTS(text=text, lang=lang)
    audio_path = "audio.mp3"
    tts.save(audio_path)
    return audio_path

# Função para adicionar texto a uma imagem usando Pillow
def add_text_to_image(image_path, text):
    # Carregar a imagem
    image = Image.open(image_path).convert("RGBA")

    # Criar uma nova imagem para o texto
    txt = Image.new('RGBA', image.size, (255, 255, 255, 0))

    # Criar um objeto de desenho
    d = ImageDraw.Draw(txt)

    # Definir a fonte (use uma fonte disponível em seu sistema ou forneça o caminho)
    font = ImageFont.load_default()

    # Usar textbbox para calcular o tamanho do texto
    text_bbox = d.textbbox((0, 0), text, font=font)
    text_width = text_bbox[2] - text_bbox[0]  # largura do texto
    text_height = text_bbox[3] - text_bbox[1]  # altura do texto

    # Calcular a posição para centralizar o texto
    text_x = (image.width - text_width) / 2
    text_y = image.height - text_height - 20  # 20 pixels do fundo

    # Adicionar o texto à nova imagem
    d.text((text_x, text_y), text, fill=(255, 255, 255, 255), font=font)

    # Combinar a imagem original com a imagem do texto
    combined = Image.alpha_composite(image, txt)

    # Salvar a imagem temporariamente
    temp_image_path = "temp_image_with_text.png"
    combined.save(temp_image_path)

    return temp_image_path

# Função para criar um vídeo com imagem, áudio e legenda
def create_video_with_caption_and_audio(image_path, text, audio_path):
    # Adicionar texto à imagem
    image_with_text_path = add_text_to_image(image_path, text)

    # Carregar o áudio
    audio_clip = AudioFileClip(audio_path)

    # Criar um clipe de imagem com a duração igual à do áudio
    image_clip = ImageClip(image_with_text_path).set_duration(audio_clip.duration)

    # Adicionar o áudio ao clipe da imagem
    image_clip = image_clip.set_audio(audio_clip)

    # Salvar o vídeo em um buffer temporário
    video_path = "/tmp/temp_video.mp4"
    image_clip.write_videofile(video_path, codec='libx264', fps=24)

    return video_path

# Passo 1: Upload da imagem
image_path = upload_image()

# Passo 2: Capturar a legenda
text_caption = input("Digite o texto da legenda: ")

# Passo 3: Gerar o áudio da legenda em português
audio_path = generate_audio_from_text(text_caption)

# Passo 4: Criar o vídeo com imagem, áudio e legenda
video_path = create_video_with_caption_and_audio(image_path, text_caption, audio_path)

# Passo 5: Exibir o vídeo no Jupyter Notebook
display(Video(video_path, embed=True))


Saving WhatsApp Image 2024-09-28 at 23.21.11 (1).jpeg to WhatsApp Image 2024-09-28 at 23.21.11 (1).jpeg
Digite o texto da legenda: Mulher empreendendora cuidando das finanças
Moviepy - Building video /tmp/temp_video.mp4.
MoviePy - Writing audio in temp_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /tmp/temp_video.mp4



Moviepy - Done !
Moviepy - video ready /tmp/temp_video.mp4


Áudio e legenda

In [11]:
!apt-get install -y imagemagick

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
imagemagick is already the newest version (8:6.9.11.60+dfsg-1.3ubuntu0.22.04.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [13]:
from moviepy.config import change_settings
change_settings({"IMAGEMAGICK_BINARY": "/usr/bin/convert"})


In [19]:
import io
from moviepy.editor import ImageClip, AudioFileClip
from gtts import gTTS
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from IPython.display import Video, display
from google.colab import files  # Para Google Colab, remova se estiver em outro ambiente

# Função para adicionar texto a uma imagem usando Pillow (sem ImageMagick)
def add_text_to_image(image_path, text, font_size=40):
    # Carregar a imagem
    image = Image.open(image_path).convert("RGBA")

    # Criar uma nova imagem para o texto
    txt = Image.new('RGBA', image.size, (255, 255, 255, 0))

    # Criar um objeto de desenho
    d = ImageDraw.Draw(txt)

    # Usar a fonte padrão
    font = ImageFont.load_default()

    # Calcular a posição do texto para centralizá-lo
    text_width, text_height = d.textbbox((0, 0), text, font=font)[2:]  # Usa textbbox para obter as dimensões
    text_x = (image.width - text_width) // 2
    text_y = image.height - text_height - 20  # Ajuste para 20 pixels acima do fundo

    # Adicionar o texto à nova imagem
    d.text((text_x, text_y), text, font=font, fill=(255, 255, 255, 255))

    # Combinar a imagem original com a imagem do texto
    combined = Image.alpha_composite(image, txt)

    # Converter a imagem combinada de volta para RGB e salvar
    combined = combined.convert("RGB")
    temp_image_path = "temp_image_with_text.jpg"
    combined.save(temp_image_path)

    return temp_image_path

# Função para gerar áudio a partir do texto
def generate_audio_from_text(text, lang='pt'):
    tts = gTTS(text=text, lang=lang)
    audio_path = "audio.mp3"
    tts.save(audio_path)
    return audio_path

# Função para criar um vídeo com imagem, áudio e legenda
def create_video_with_caption_and_audio(image_path, text, audio_path, duration=5, font_size=40):
    # Adicionar legenda à imagem usando Pillow
    image_with_text_path = add_text_to_image(image_path, text, font_size=font_size)

    # Criar um clipe de imagem com a legenda
    image_clip = ImageClip(image_with_text_path).set_duration(duration)

    # Adicionar o áudio ao clipe da imagem
    audio_clip = AudioFileClip(audio_path)
    image_clip = image_clip.set_audio(audio_clip)

    # Salvar o vídeo com imagem, áudio e legenda
    output_video_path = "output_video.mp4"
    image_clip.write_videofile(output_video_path, codec='libx264', fps=24)

    return output_video_path

# Função para exibir o vídeo no Jupyter Notebook (se necessário)
def display_video(video_path):
    display(Video(video_path, embed=True))

# Função para fazer o upload de uma imagem
def upload_image():
    uploaded = files.upload()  # Abre um diálogo de upload de arquivo no Colab
    image_path = next(iter(uploaded))  # Pega o caminho do primeiro arquivo carregado
    return image_path

# Etapas principais
# 1. Fazer upload da imagem
image_path = upload_image()  # Isso abrirá o diálogo para você selecionar a imagem

# 2. Capturar a legenda
text_caption = input("Digite o texto da legenda: ")

# 3. Gerar o áudio da legenda
audio_path = generate_audio_from_text(text_caption)

# 4. Criar o vídeo com imagem, áudio e legenda
output_video_path = create_video_with_caption_and_audio(image_path, text_caption, audio_path)

# 5. Exibir o vídeo (opcional)
display_video(output_video_path)

Saving WhatsApp Image 2024-09-28 at 23.21.11 (1).jpeg to WhatsApp Image 2024-09-28 at 23.21.11 (1) (3).jpeg
Digite o texto da legenda: Mulher
Moviepy - Building video output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4


In [3]:
!pip install gtts

In [ ]:
import moviepy.config as mpy_config
mpy_config.IMAGEMAGICK_BINARY = "/usr/bin/convert"  # Verifique o caminho

In [ ]:
pip install --upgrade moviepy

In [ ]:
!magick -version

/bin/bash: line 1: magick: command not found


In [ ]:
!apt-get update
!apt-get install -y imagemagick

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,584 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,156 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,352 kB]
Hit:13 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:14 http://archive.ub

In [ ]:
!apt-get install -y imagemagick


In [ ]:
!apt-get install imagemagick

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
imagemagick is already the newest version (8:6.9.11.60+dfsg-1.3ubuntu0.22.04.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [27]:
import io
from moviepy.editor import ImageClip, AudioFileClip
from gtts import gTTS
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from IPython.display import Video, display
from google.colab import files  # Para Google Colab, remova se estiver em outro ambiente

# Função para adicionar texto a uma imagem usando Pillow (sem ImageMagick)
def add_text_to_image(image_path, text, font_size=180):  # Aumentar o tamanho da fonte
    # Carregar a imagem
    image = Image.open(image_path).convert("RGBA")

    # Criar uma nova imagem para o texto
    txt = Image.new('RGBA', image.size, (255, 255, 255, 0))

    # Criar um objeto de desenho
    d = ImageDraw.Draw(txt)

    # Usar a fonte padrão
    font = ImageFont.load_default()  # Você pode substituir isso por uma fonte específica, se desejar

    # Calcular a posição do texto para centralizá-lo
    text_width, text_height = d.textbbox((0, 0), text, font=font)[2:]  # Usa textbbox para obter as dimensões
    text_x = (image.width - text_width) // 2

    # Ajustar a posição vertical para mover o texto a 100 pixels do fundo
    text_y = image.height - text_height - 100  # Agora definido para ficar 100 pixels acima do fundo

    # Adicionar um contorno ao texto para melhor legibilidade
    outline_range = 2  # Largura do contorno
    # Desenhar contorno
    for dx in range(-outline_range, outline_range + 1):
        for dy in range(-outline_range, outline_range + 1):
            if dx != 0 or dy != 0:
                d.text((text_x + dx, text_y + dy), text, font=font, fill=(0, 0, 0, 255))  # Contorno preto

    # Adicionar o texto à nova imagem
    d.text((text_x, text_y), text, font=font, fill=(255, 255, 255, 255))  # Texto branco

    # Combinar a imagem original com a imagem do texto
    combined = Image.alpha_composite(image, txt)

    # Converter a imagem combinada de volta para RGB e salvar
    combined = combined.convert("RGB")
    temp_image_path = "temp_image_with_text.jpg"
    combined.save(temp_image_path)

    return temp_image_path

# Função para gerar áudio a partir do texto
def generate_audio_from_text(text, lang='pt'):
    tts = gTTS(text=text, lang=lang)
    audio_path = "audio.mp3"
    tts.save(audio_path)
    return audio_path

# Função para criar um vídeo com imagem, áudio e legenda
def create_video_with_caption_and_audio(image_path, text, audio_path, duration=5, font_size=180):  # Aumentar o tamanho da fonte aqui também
    # Adicionar legenda à imagem usando Pillow
    image_with_text_path = add_text_to_image(image_path, text, font_size=font_size)

    # Criar um clipe de imagem com a legenda
    image_clip = ImageClip(image_with_text_path).set_duration(duration)

    # Adicionar o áudio ao clipe da imagem
    audio_clip = AudioFileClip(audio_path)
    image_clip = image_clip.set_audio(audio_clip)

    # Salvar o vídeo com imagem, áudio e legenda
    output_video_path = "output_video.mp4"
    image_clip.write_videofile(output_video_path, codec='libx264', fps=24)

    return output_video_path

# Função para exibir o vídeo no Jupyter Notebook (se necessário)
def display_video(video_path):
    display(Video(video_path, embed=True))

# Função para fazer o upload de uma imagem
def upload_image():
    uploaded = files.upload()  # Abre um diálogo de upload de arquivo no Colab
    image_path = next(iter(uploaded))  # Pega o caminho do primeiro arquivo carregado
    return image_path

# Etapas principais
# 1. Fazer upload da imagem
image_path = upload_image()  # Isso abrirá o diálogo para você selecionar a imagem

# 2. Capturar a legenda
text_caption = input("Digite o texto da legenda: ")

# 3. Gerar o áudio da legenda
audio_path = generate_audio_from_text(text_caption)

# 4. Criar o vídeo com imagem, áudio e legenda
output_video_path = create_video_with_caption_and_audio(image_path, text_caption, audio_path)

# 5. Exibir o vídeo (opcional)
display_video(output_video_path)

Saving WhatsApp Image 2024-09-28 at 23.21.11 (1).jpeg to WhatsApp Image 2024-09-28 at 23.21.11 (1) (9).jpeg
Digite o texto da legenda: Mulher empreendedora cuidando do negócio
Moviepy - Building video output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4
